# ***Server Side***

# <a id='index'>Index</a>

+ <a href='#connection'>1. Connecting Client and Server</a>
+ <a href='#server_keys'>2. Generating Server Keys</a>
+ <a href='#exchange'>3. Exchange of Public Keys for encrypted communication</a>
    + <a href='#client_keys'>3.1 Receiving Client Public Keys from Client</a>
    + <a href='#sending_server_keys_to_client'>3.2 Sending Server Public Keys to Client</a>
+ <a href='#communicate'>5. Encrypted Communication</a>

In [2]:
import socket
import numpy as np
import gmpy2
from gmpy2 import powmod,mpz,isqrt,invert
import pyecm
from el_gamal import generate_keys,get_blocks,format_plain_text,format_decrypted_plain_text,encrypt,decrypt
#from rsa import generate_keys,get_blocks,format_plain_text,format_decrypted_plain_text,encrypt,decrypt

# <a id='connection'>1. Connecting Client and Server</a>
<a href='#index'>Go back to top</a>

In [3]:
host = 'localhost'        # Symbolic name meaning all available interfaces
port = 12345     # Arbitrary non-privileged port
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((host, port))
s.listen(1)
conn, addr = s.accept()
print('Connected by', addr)

Connected by ('127.0.0.1', 57844)


# <a id='server_keys'>2. Generating Server Keys</a>
<a href='#index'>Go back to top</a>

In [4]:
server_e1,server_e2,server_p,server_d = generate_keys()
print('--------------------------------GENERATED BY SERVER---------------------------------------------------')
print('Server public keys : e1 :',server_e1,'e2 :',server_e2,'p :',server_p)
print('Server private key : d :',server_d)
print('-----------------------------------------------------------------------------------')

--------------------------------GENERATED BY SERVER---------------------------------------------------
Server public keys : e1 : 24 e2 : 52405865866852881362164 p : 1000000000000000000144879
Server private key : d : 35
-----------------------------------------------------------------------------------


# <a id='exchange'>3. Exchange of Public Keys for encrypted communication</a>
<a href='#index'>Go back to top</a>

## <a id='client_keys'>3.1 Receiving Client Public Keys from Client</a>

In [5]:
client_public_keys = conn.recv(1024)
client_public_keys = (client_public_keys.decode('utf-8')).split(" ")
client_public_keys[0] = int(client_public_keys[0])
client_public_keys[1] = int(client_public_keys[1])
client_public_keys[2] = int(client_public_keys[2])
client_public_keys = (client_public_keys[0],client_public_keys[1],client_public_keys[2])
print('Client Public Keys sent by Client: e1 :',client_public_keys[0],'e2 :',client_public_keys[1],'p :',client_public_keys[2])

Client Public Keys sent by Client: e1 : 34 e2 : 746862570415608040926448 p : 1000000000000000000089817


## <a id='sending_server_keys_to_client'>3.2 Sending Server Public Keys to Client</a>

In [6]:
server_public_keys = str(server_e1) + ' ' + str(server_e2) + ' ' + str(server_p)
conn.sendall((server_public_keys).encode())

# <a id='communicate'>4. Encrypted Communication</a>
<a href='#index'>Go back to top</a>

In [7]:
print('\n------------------------------MESSAGES BETWEEN CLIENT AND SERVER ARE NOW ENE-TO-END ENCRYPTED---------------------------')
while True:
    cipher_text_blocks = []
    while True :
        data = ''
        data = conn.recv(1024)
        data = data.decode('utf-8')
        if data == 'END':
            break
        data = data.split(" ")
        cipher_text_blocks.append((mpz(int(data[0])),mpz(int(data[1]))))
    decypted_plain_text_blocks = decrypt(cipher_text_blocks,server_d,(server_e1,server_e2,server_p))
    plain_text_after_decryption = format_decrypted_plain_text(decypted_plain_text_blocks)
    print('Client :',plain_text_after_decryption)
    if plain_text_after_decryption == 'BYE':
        break
    
    PT = input('Server : ')
    PT = get_blocks(PT)
    plain_text_blocks = format_plain_text(PT)
    cipher_text_blocks = encrypt(plain_text_blocks,client_public_keys)
    for block in cipher_text_blocks:
        temp = str(block[0]) + ' ' + str(block[1])
        conn.sendall(temp.encode())
    temp = 'END'
    conn.sendall(temp.encode())
    if PT[0] == 'BYE':
        break
conn.close()


------------------------------MESSAGES BETWEEN CLIENT AND SERVER ARE NOW ENE-TO-END ENCRYPTED---------------------------
Client : Hello server!! This is client!!!
Server : Hello client!! This is server!!!
Client : Good to hear from you!!!
Server : What is our port of communication?
Client : It's 12345
Server : Thanks :)
Client : BYE
